In [17]:
from pathlib import Path
import cv2

sift = cv2.SIFT_create(
    nOctaveLayers=5,
    sigma=1.414
)


def load_logos(base_folder: str):
    logos_dict = {}
    base_path = Path(base_folder)

    for brand_folder in base_path.iterdir():
        if brand_folder.is_dir():
            brand_name = brand_folder.name
            logos = [str(file) for file in brand_folder.glob("*") if file.is_file()]
            logos_dict[brand_name] = logos

    return logos_dict

folder = "Logos" 
logos_paths = load_logos(folder)

print(logos_paths)
descriptores = {}


for brand, paths in logos_paths.items():
    descriptores[brand] = []
    for path in paths:
        logo = cv2.imread(path)
        
        if logo is None:
            print(f"Error loading image: {path}")
            continue

        gray_logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
        kp_logo, des_logo = sift.detectAndCompute(logo, None)
        descriptores[brand].append((kp_logo, des_logo))



{'Adidas': ['Logos\\Adidas\\logo1.png'], 'Nike': ['Logos\\Nike\\logo1.png', 'Logos\\Nike\\logo10.png', 'Logos\\Nike\\logo2.png', 'Logos\\Nike\\logo3.png', 'Logos\\Nike\\logo4.png', 'Logos\\Nike\\logo5.png', 'Logos\\Nike\\logo6.png', 'Logos\\Nike\\logo7.png', 'Logos\\Nike\\logo8.png', 'Logos\\Nike\\logo9.png'], 'Puma': ['Logos\\Puma\\logo1.png']}


In [15]:
import cv2
import numpy as np
from pathlib import Path

# Load all cloth images from the Cloth folder
cloth_folder = Path("Cloth")
cloth_images = list(cloth_folder.glob("*"))

# Process each cloth image
for cloth_path in cloth_images:
    if not cloth_path.is_file():
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {cloth_path.name}")
    print('='*60)
    
    shirt = cv2.imread(str(cloth_path), cv2.IMREAD_GRAYSCALE)
    
    if shirt is None:
        print(f"Error loading image: {cloth_path}")
        continue
    
    # Compute SIFT features for the shirt
    kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)
    
    if des_shirt is None:
        print("No descriptors found in shirt")
        continue
    
    best_brand = None
    best_match_count = 0
    best_matches = None
    best_logo_kp = None
    best_H = None
    best_mask = None
    
    # Match against all logos from all brands
    for brand, logo_data in descriptores.items():
        for idx, (kp_logo, des_logo) in enumerate(logo_data):
            if des_logo is None:
                continue
            
            # Match with KNN
            bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
            knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)
            
            # Lowe's ratio test
            good = []
            ratio_thresh = 0.75
            for match_pair in knn_matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_thresh * n.distance:
                        good.append(m)
            
            # Check if this is the best match so far
            if len(good) > best_match_count:
                best_match_count = len(good)
                best_brand = brand
                best_matches = good
                best_logo_kp = kp_logo
                
                # Compute homography if enough matches
                MIN_MATCH_COUNT = 4
                if len(good) >= MIN_MATCH_COUNT:
                    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                    best_H = H
                    best_mask = mask
    
    # Display results for this cloth
    print(f"Best match: {best_brand} with {best_match_count} good matches")
    
    if best_matches and best_match_count >= 4:
        # Load the best matching logo for visualization
        logo_path = logos_paths[best_brand][0]
        logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
        
        matchesMask = best_mask.ravel().tolist() if best_mask is not None else None
        
        # Draw matches
        result = cv2.drawMatches(
            logo, best_logo_kp,
            shirt, kp_shirt,
            best_matches, None,
            matchesMask=matchesMask,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        cv2.namedWindow(f"Matches - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Matches - {cloth_path.name}", result)
        cv2.resizeWindow(f"Matches - {cloth_path.name}", 1200, 800)
        
        # Draw bounding box on shirt
        if best_H is not None:
            h, w = logo.shape
            logo_corners = np.float32([
                [0, 0],
                [0, h - 1],
                [w - 1, h - 1],
                [w - 1, 0]
            ]).reshape(-1, 1, 2)
            
            projected_corners = cv2.perspectiveTransform(logo_corners, best_H)
            
            shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
            shirt_with_box = cv2.polylines(
                shirt_color,
                [np.int32(projected_corners)],
                isClosed=True,
                color=(0, 255, 0),
                thickness=3,
                lineType=cv2.LINE_AA
            )
            
            cv2.namedWindow(f"Detection - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"Detection - {cloth_path.name}", shirt_with_box)
            cv2.resizeWindow(f"Detection - {cloth_path.name}", 800, 800)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print(f"Not enough matches found for detection")


Processing: nike_shirt1.png
Best match: Nike with 74 good matches

Processing: nike_shirt2.png
Best match: Nike with 213 good matches


In [ ]:
import cv2
import numpy as np

# Read image
img = cv2.imread("./R.jpeg")
Z = img.reshape((-1, 3))          # shape: (num_pixels, 3)
Z = np.float32(Z)

# Define criteria and number of clusters (K)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4  # number of segments
attempts = 10


filtered = cv2.pyrMeanShiftFiltering(img, sp=20, sr=40)

cv2.imshow("Original", img)
cv2.imshow("MeanShift Filtered", filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()

gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)

cv2.imshow("Threshold after MeanShift", thresh)0
cv2.waitKey(0)
cv2.destroyAllWindows()

Good matches: 6
Not enough matches for RANSAC: 6 / 10


In [2]:
import cv2
import numpy as np

# ---------- 1) Load images ----------
logo = cv2.imread("./licensed-image.jpeg", cv2.IMREAD_GRAYSCALE)

# Read shirt in color (for segmentation) and gray (for SIFT)
shirt_color = cv2.imread("./shirts/image.png")
shirt_gray = cv2.cvtColor(shirt_color, cv2.COLOR_BGR2GRAY)

if logo is None or shirt_color is None:
    print("Error loading images")
    exit()

# ---------- 2) Segment shirt with MeanShift + threshold ----------
# MeanShift smoothing (color-based segmentation)
filtered = cv2.pyrMeanShiftFiltering(shirt_color, sp=10, sr=10)

# Convert to gray and threshold (Otsu) to get binary mask
filtered_gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(filtered_gray, 0, 255, cv2.THRESH_OTSU)

# Optional: clean mask a bit with morphology
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

cv2.imshow("MeanShift Filtered", filtered)
cv2.imshow("Segmentation Mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---------- 3) SIFT with custom params ----------
sift = cv2.SIFT_create(
    nOctaveLayers=7,
    sigma=1.414
)

# Logo: no mask
kp_logo, des_logo = sift.detectAndCompute(logo, None)

# Shirt: use segmentation mask to restrict keypoints
kp_shirt, des_shirt = sift.detectAndCompute(shirt_gray, mask)

if des_logo is None or des_shirt is None:
    print("No descriptors found – maybe the mask removed too much?")
    exit()

# ---------- 4) KNN matching + Lowe's ratio ----------
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)

good = []
ratio_thresh = 0.75
for m, n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good.append(m)

print("Good matches:", len(good))

MIN_MATCH_COUNT = 0  # tweak as you like

if len(good) >= MIN_MATCH_COUNT:
    # ---------- 5) Prepare points for RANSAC homography ----------
    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # ---------- 6) Compute homography with RANSAC ----------
    H, mask_inliers = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask_inliers.ravel().tolist() if mask_inliers is not None else None

    print("Homography:\n", H)

    # ---------- 7) Draw inlier matches ----------
    result = cv2.drawMatches(
        logo, kp_logo,
        shirt_gray, kp_shirt,
        good, None,
        matchesMask=matchesMask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    cv2.namedWindow("RANSAC Matches (Segmented Shirt)", cv2.WINDOW_NORMAL)
    cv2.imshow("RANSAC Matches (Segmented Shirt)", result)
    cv2.resizeWindow("RANSAC Matches (Segmented Shirt)", 1200, 800)

    # ---------- 8) Project logo bounding box onto shirt ----------
    if H is not None:
        h_logo, w_logo = logo.shape

        logo_corners = np.float32([
            [0, 0],
            [0, h_logo - 1],
            [w_logo - 1, h_logo - 1],
            [w_logo - 1, 0]
        ]).reshape(-1, 1, 2)

        projected_corners = cv2.perspectiveTransform(logo_corners, H)

        # Draw on color shirt image
        shirt_with_box = shirt_color.copy()
        shirt_with_box = cv2.polylines(
            shirt_with_box,
            [np.int32(projected_corners)],
            isClosed=True,
            color=(0, 255, 0),
            thickness=3,
            lineType=cv2.LINE_AA
        )

        cv2.namedWindow("Logo on Segmented Shirt", cv2.WINDOW_NORMAL)
        cv2.imshow("Logo on Segmented Shirt", shirt_with_box)
        cv2.resizeWindow("Logo on Segmented Shirt", 800, 800)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print(f"Not enough matches for RANSAC: {len(good)} / {MIN_MATCH_COUNT}")


Good matches: 49
Homography:
 [[-1.34259519e-01 -4.72588672e-03  3.88532264e+02]
 [-1.08757145e-01 -3.83178548e-03  3.14713580e+02]
 [-3.45558284e-04 -1.21815764e-05  1.00000000e+00]]
